In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.callbacks import ModelCheckpoint
# from keras import backend as K
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split

# Set paths and read dataset
file_path = 'fer2013.csv'
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
names = ['emotion', 'pixels', 'usage']
df = pd.read_csv(file_path, names=names, na_filter=False)
df = df[1:]  # Remove the header row from the data
df.head()

# Preprocess the data
def getData(file_path):
    Y, X = [], []
    first = True
    for line in open(file_path):
        if first:
            first = False
            continue
        row = line.split(',')
        Y.append(int(row[0]))
        X.append([int(p) for p in row[1].split()])
    X = np.array(X) / 255.0  # Normalize
    Y = np.array(Y)
    return X, Y

X, Y = getData(file_path)
X = X.reshape(-1, 48, 48, 1)  # reshape for CNN
num_classes = len(set(Y))

# One-hot encoding
y_onehot = (np.arange(num_classes) == Y[:, None]).astype(np.float32)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.1, random_state=10)

# Define the CNN model
def my_model():
    model = Sequential()
    input_shape = (48, 48, 1)

    model.add(Conv2D(64, (5, 5), activation='relu', padding='same', input_shape=input_shape))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (5, 5), activation='relu', padding='same'))
    model.add(Conv2D(128, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Prepare model and training
model = my_model()
model.summary()

# Save model after each epoch
model_save_path = 'model_filter.h5'
checkpoint = ModelCheckpoint(filepath=model_save_path, save_best_only=True)

# Clear session and reset optimizer learning rate
K.clear_session()
model = my_model()
model.optimizer.learning_rate.assign(0.0005)
# Train the model
history = model.fit(
    X_train, y_train,
    batch_size=64,
    epochs=20,
    verbose=1,
    validation_data=(X_test, y_test),
    shuffle=True,
    callbacks=[checkpoint]
)
